<a href="https://colab.research.google.com/github/shelke16/HPC/blob/main/bubble_merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile merge.cpp
#include<iostream>
#include<stdlib.h>
#include<omp.h>
using namespace std;

void mergesort(int a[],int i,int j);
void merge(int a[],int i1,int j1,int i2,int j2);

void mergesort(int a[],int i,int j)
{
    int mid;
    if(i<j)
    {
        mid=(i+j)/2;

        #pragma omp parallel sections
        {

            #pragma omp section
            {
                mergesort(a,i,mid);
            }

            #pragma omp section
            {
                mergesort(a,mid+1,j);
            }
        }

        merge(a,i,mid,mid+1,j);
    }

}

void merge(int a[],int i1,int j1,int i2,int j2)
{
    int temp[1000];
    int i,j,k;
    i=i1;
    j=i2;
    k=0;

    while(i<=j1 && j<=j2)
    {
        if(a[i]<a[j])
        {
            temp[k++]=a[i++];
        }
        else
        {
            temp[k++]=a[j++];
 }
    }

    while(i<=j1)
    {
        temp[k++]=a[i++];
    }

    while(j<=j2)
    {
        temp[k++]=a[j++];
    }

    for(i=i1,j=0;i<=j2;i++,j++)
    {
        a[i]=temp[j];
    }
}


int main()
{
    int *a,n,i;
    double start_time, end_time, seq_time, par_time;

    cout<<"\n enter total no of elements=>";
    cin>>n;
    a= new int[n];

    cout<<"\n enter elements=>";
    for(i=0;i<n;i++)
    {
        cin>>a[i];
    }

    // Sequential algorithm
    start_time = omp_get_wtime();
    mergesort(a, 0, n-1);
    end_time = omp_get_wtime();
    seq_time = end_time - start_time;
    cout << "\nSequential Time: " << seq_time << endl;

    // Parallel algorithm
    start_time = omp_get_wtime();
    #pragma omp parallel
    {
        #pragma omp single
        {
            mergesort(a, 0, n-1);
        }
}
end_time = omp_get_wtime();
par_time = end_time - start_time;
cout << "\nParallel Time: " << par_time << endl;
cout<<"\n sorted array is=>";
for(i=0;i<n;i++)
{
cout<<"\n"<<a[i];
}
return 0;
}

Writing merge.cpp


In [2]:
!g++ merge.cpp -fopenmp -o merge
!./merge


 enter total no of elements=>4

 enter elements=>5
7
8
9

Sequential Time: 0.000150277

Parallel Time: 4.888e-06

 sorted array is=>
5
7
8
9

In [3]:
import multiprocessing
import time

def merge(arr, l, m, r):
    n1 = m - l + 1
    n2 = r - m

    L = arr[l:l + n1]
    R = arr[m + 1:m + 1 + n2]

    i = 0
    j = 0
    k = l

    while i < n1 and j < n2:
        if L[i] < R[j]:
            arr[k] = L[i]
            i += 1
        else:
            arr[k] = R[j]
            j += 1
        k += 1

    while i < n1:
        arr[k] = L[i]
        i += 1
        k += 1

    while j < n2:
        arr[k] = R[j]
        j += 1
        k += 1

def mergesort(arr, l, r):
    if l < r:
        m = (l + r) // 2
        mergesort(arr, l, m)
        mergesort(arr, m + 1, r)
        merge(arr, l, m, r)

def parallel_mergesort(arr, l, r):
    if l < r:
        if r - l < 500:  # Threshold: for small array, do sequential sort
            mergesort(arr, l, r)
        else:
            m = (l + r) // 2
            left_proc = multiprocessing.Process(target=parallel_mergesort, args=(arr, l, m))
            right_proc = multiprocessing.Process(target=parallel_mergesort, args=(arr, m + 1, r))

            left_proc.start()
            right_proc.start()

            left_proc.join()
            right_proc.join()

            merge(arr, l, m, r)

if __name__ == "__main__":
    n = int(input("\nEnter total number of elements: "))
    arr = []
    print("\nEnter elements:")
    for _ in range(n):
        arr.append(int(input()))

    arr_seq = arr.copy()
    arr_par = arr.copy()

    # Sequential Merge Sort
    start_time = time.time()
    mergesort(arr_seq, 0, n-1)
    end_time = time.time()
    seq_time = end_time - start_time
    print("\nSequential Time:", seq_time)

    # Parallel Merge Sort
    start_time = time.time()
    parallel_mergesort(arr_par, 0, n-1)
    end_time = time.time()
    par_time = end_time - start_time
    print("\nParallel Time:", par_time)

    print("\nSorted Array:")
    for elem in arr_par:
        print(elem)



Enter total number of elements: 6

Enter elements:
9
4
76
87
21
34

Sequential Time: 1.8358230590820312e-05

Parallel Time: 1.9073486328125e-05

Sorted Array:
4
9
21
34
76
87


In [4]:
%%writefile bubble.cpp
#include <iostream>
#include <omp.h>
using namespace std;
void sequentialBubbleSort(int *, int);
void parallelBubbleSort(int *, int);
void swap(int &, int &);
void sequentialBubbleSort(int *a, int n)
{
int swapped;
for (int i = 0; i < n; i++)
{
swapped = 0;
for (int j = 0; j < n - 1; j++)
{
if (a[j] > a[j + 1])
{
swap(a[j], a[j + 1]);
swapped = 1;
}
}
if (!swapped)
break;
}
}
void parallelBubbleSort(int *a, int n)
{
int swapped;
for (int i = 0; i < n; i++)
{
swapped = 0;
int first=i%2;
#pragma omp parallel for shared(a,first)
for (int j = first; j < n - 1; j++)
{
if (a[j] > a[j + 1])
{
swap(a[j], a[j + 1]);
swapped = 1;
}
}
if (!swapped)
            break;
    }
}

void swap(int &a, int &b)
{
    int test;
    test = a;
    a = b;
    b = test;
}

int main()
{
    int *a, n;
    cout << "\n enter total no of elements=>";
    cin >> n;
    a = new int[n];
    cout << "\n enter elements=>";
    for (int i = 0; i < n; i++)
    {
        cin >> a[i];
    }

    double start_time = omp_get_wtime(); // start timer for sequential algorithm
    sequentialBubbleSort(a, n);
    double end_time = omp_get_wtime(); // end timer for sequential algorithm

    cout << "\n sorted array is=>";
    for (int i = 0; i < n; i++)
    {
        cout << a[i] << endl;
    }

    cout << "Time taken by sequential algorithm: " << end_time - start_time << " seconds" << endl;

    start_time = omp_get_wtime(); // start timer for parallel algorithm
    parallelBubbleSort(a, n);
    end_time = omp_get_wtime(); // end timer for parallel algorithm

    cout << "\n sorted array is=>";
    for (int i = 0; i < n; i++)
    {
        cout << a[i] << endl;
    }

    cout << "Time taken by parallel algorithm: " << end_time - start_time << " seconds" << endl;

    delete[] a; // Don't forget to free the allocated memory

    return 0;
}

Writing bubble.cpp


In [5]:
!g++ bubble.cpp -fopenmp -o bubble
!./bubble


 enter total no of elements=>5

 enter elements=>34
78
45
23
12

 sorted array is=>12
23
34
45
78
Time taken by sequential algorithm: 1.004e-06 seconds

 sorted array is=>12
23
34
45
78
Time taken by parallel algorithm: 0.00329675 seconds


In [6]:
import multiprocessing
import time

def swap(arr, i, j):
    arr[i], arr[j] = arr[j], arr[i]

def sequential_bubble_sort(arr, n):
    for i in range(n):
        swapped = False
        for j in range(n - 1):
            if arr[j] > arr[j + 1]:
                swap(arr, j, j + 1)
                swapped = True
        if not swapped:
            break

def parallel_bubble_sort(arr, n):
    for i in range(n):
        swapped = multiprocessing.Value('i', 0)  # Shared variable for swapping
        first = i % 2

        def process_chunk(start):
            local_swapped = False
            for j in range(start, n - 1, 2):
                if arr[j] > arr[j + 1]:
                    swap(arr, j, j + 1)
                    local_swapped = True
            if local_swapped:
                swapped.value = 1

        jobs = []
        for start in range(first, first + 1):
            p = multiprocessing.Process(target=process_chunk, args=(start,))
            p.start()
            jobs.append(p)

        for job in jobs:
            job.join()

        if not swapped.value:
            break

if __name__ == "__main__":
    n = int(input("\nEnter total number of elements: "))
    a = []
    print("\nEnter elements:")
    for _ in range(n):
        a.append(int(input()))

    arr_seq = a.copy()
    arr_par = a.copy()

    # Sequential Bubble Sort
    start_time = time.time()
    sequential_bubble_sort(arr_seq, n)
    end_time = time.time()
    print("\nSorted array (Sequential):")
    for num in arr_seq:
        print(num)
    print("Time taken by sequential algorithm:", end_time - start_time, "seconds")

    # Parallel Bubble Sort
    start_time = time.time()
    parallel_bubble_sort(arr_par, n)
    end_time = time.time()
    print("\nSorted array (Parallel):")
    for num in arr_par:
        print(num)
    print("Time taken by parallel algorithm:", end_time - start_time, "seconds")



Enter total number of elements: 4

Enter elements:
23
45
67
34

Sorted array (Sequential):
23
34
45
67
Time taken by sequential algorithm: 1.2874603271484375e-05 seconds

Sorted array (Parallel):
23
45
67
34
Time taken by parallel algorithm: 0.02559804916381836 seconds
